In [8]:
import open3d as o3d
import numpy as np

In [9]:
def compute_darboux_frame_for_points(pcd: o3d.geometry.PointCloud, sample_points: o3d.geometry.PointCloud, search_radius: float = 0.02):
    """
    Compute the Darboux frame for each point in pcd using sample_points.

    Parameters
    ----------
    pcd : o3d.geometry.PointCloud
        The point cloud for which to compute the Darboux frame.
    sample_points : o3d.geometry.PointCloud
        The point cloud used for sampling to compute the Darboux frame.
    search_radius : float, optional
        The radius for neighborhood search used in PCA, by default 0.02[m].
    
    Returns
    -------
    darboux_frames : list
        A list of rotation matrices representing the Darboux frame for each point in pcd.
    """

    kdtree = o3d.geometry.KDTreeFlann(pcd)
    points = np.asarray(pcd.points)
    sampled = np.asarray(sample_points.points)
    rotation_matrices = []
    frames = []

    for p in sampled:
        [k, idx, _] = kdtree.search_radius_vector_3d(p, search_radius)
        if k < 3:
            frames.append(None) # Not enough neighbors to compute the frame
            print(f'||| warn: Point {p} has less than 3 neighbors, skipping.')
            continue

        neighbors = points[idx]
        # Compute the covariance matrix(centered)
        centered = neighbors - np.mean(neighbors, axis=0)
        cov = centered.T @ centered

        # Compute the eigenvalues(ascending order) and eigenvectors(normalized) 
        eigvals, eigvecs = np.linalg.eigh(cov)

        # Define the Darboux frame
        normal = eigvecs[:, 0] # Normal (Minimum eigenvalue)
        tangent1 = eigvecs[:, 2] # Principal curvature direction (Maximum eigenvalue)
        tangent2 = eigvecs[:, 1] # The other principal curvature direction (Middle eigenvalue)

        # Ensure normal looks to origin (assuming the sensor is at the origin (0, 0, 0))
        if np.dot(normal, p) < 0:
            normal = -normal

        # Ensure right-handed coordinate system
        if np.dot(np.cross(tangent1, tangent2), normal) < 0:
            tangent2 = -tangent2 # return tangent2 to become right-handedness

        # Create the rotation matrix
        rotation_matrix = np.column_stack((tangent1, tangent2, normal)) # x, y, z axes
        T = np.eye(4)
        T[:3, :3] = rotation_matrix
        T[:3, 3] = p # Set the translation to the point p
        frames.append(T)

    return frames

In [10]:
# load point cloud
pcd = o3d.io.read_point_cloud('../data/segmented/remote_controller.ply')
sampled_pcd = o3d.io.read_point_cloud('../data/segmented/remote_controller_random_sampled.ply')
# compute the Darboux frame
frames = compute_darboux_frame_for_points(pcd, sampled_pcd)

In [11]:
# visualize
visuals = []
for T in frames:
    if T is not None:
        frame_axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.01)
        #frame.rotate(R, center=(0, 0, 0))  # Rotate the frame to match the Darboux frame
        frame_axes.transform(T)
        visuals.append(frame_axes)
origin = o3d.geometry.TriangleMesh.create_coordinate_frame(0.05)

o3d.visualization.draw_geometries(visuals+[pcd]+[origin], window_name='Darboux Frame Visualization', mesh_show_back_face=True)

In [12]:
from scipy.spatial.transform import Rotation as R

In [13]:
def generate_hand_pose_candidates(base_frame: np.ndarray, y_offsets: np.ndarray | list, phi_angles: np.ndarray | list):
    """Generate grasp hand pose *h* by applying a grid of offsets/rotations to the base frame.

    Parameters
    ----------
    base_frame : np.ndarray
        Homogeneous transform that represents the Darboux frame located at the sample point.
        Columns correnspond to **x (approach)**, **y (closing)**, **z (normal)**.
    y_offsets : np.ndarray | list
        Offsets along the y-axis (closing direction) to apply to the base frame.
    phi_angles : np.ndarray | list
        Angles in radians to rotate the base frame around the z-axis (normal direction).
    
    Returns
    -------
    list[np.ndarray]
        List of homogeneous transforms representing the generated hand poses.

    """

    y_offsets = np.asarray(y_offsets).astype(float)
    phi_angles = np.asarray(phi_angles).astype(float)

    hand_poses = []
    for y_offset in y_offsets:
        for phi in phi_angles:
            transform = np.eye(4)
            transform[:3, :3] = R.from_euler('z', phi).as_matrix()  # Rotate around z-axis
            transform[:3, 3] = [0.0, y_offset, 0.0]
            hand_pose_candidate = base_frame @ transform  # Apply the transformation to the base frame
            hand_poses.append(hand_pose_candidate)
    
    return hand_poses


In [14]:
y_grids = np.linspace(-0.02, 0.02, 8)  # y offsets for the hand pose
phi_grids = np.linspace(-np.pi/2, np.pi/2, 8)  # phi angles for the hand pose
poses = generate_hand_pose_candidates(frames[0], y_grids, phi_grids)

for pose in poses:
    frame_axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.01)
    frame_axes.transform(pose)  # Transform the frame to the hand pose
    visuals.append(frame_axes)
o3d.visualization.draw_geometries(visuals+[pcd]+[origin], window_name='Hand Pose Candidates', mesh_show_back_face=True)

In [15]:
import pickle as pkl
import os

os.makedirs('../data/darboux_frames', exist_ok=True)
with open('../data/darboux_frames/candidate_poses.pkl', 'wb') as f:
    pkl.dump(poses, f)